In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print('os')
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

os
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [2]:
#print output
def out(preds):
    output = pd.DataFrame({'PassengerId': test.PassengerId,'Survived': preds})
    output.to_csv('submission.csv', index=False)

In [3]:
import pandas as pd
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")
test = pd.read_csv("../input/titanic/test.csv")
train = pd.read_csv("../input/titanic/train.csv")
rstate=30
t_size=0.3

In [4]:
#data preprocessor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
def process_data(data):
    encode = LabelEncoder()
    ss=StandardScaler()
    encode_features=['Sex','Cabin']
    data.SibSp=data.SibSp.fillna(0)+data.Parch.fillna(0)
    #Sib_data.SibSp='SibPa'
    embarked_encode={'S':1,'C':3,'Q':2,'N':0}
    name_encoded={ 'Don':0, 'Rev':0, 'Sir':0, 'Capt':0, 'Jonkheer':0, 'Major':1, 'Col':1, 'Dr':2, 'Master':3, 'Mr':4, 'Miss':5, 'Mrs':6, 'Mme':7, 'Mlle':7, 'the Countess':7, 'Lady':7 ,'Ms':7}
    data.Embarked=[embarked_encode[x] for x in data.Embarked.fillna('N')]
    data.Name=[name_encoded.get(y,0) for y in data.Name.apply(lambda x:x.split(',')[1].split('.')[0].strip())]
    data_updated=data[data.columns.drop(['Parch','Ticket','PassengerId'])]
    #data_updated=data_updated[data_updated.columns.drop('Embarked')].join(data_updated.Embarked.fillna('N'))
    data_updated=data_updated[data_updated.columns.drop('Age')].join(data_updated.Age.fillna(0))
    data_updated.Cabin=data_updated.Cabin.fillna('U').apply(lambda x:x[0])
    data_updated= data_updated[data_updated.columns.drop(encode_features)].join(data_updated[encode_features].apply(encode.fit_transform))
    data_updated.Cabin=ss.fit_transform(data_updated.Cabin.values.reshape(-1,1))
    data_updated.Name=ss.fit_transform(data_updated.Name.values.reshape(-1,1))
    data_updated.Embarked=ss.fit_transform(data_updated.Embarked.values.reshape(-1,1))
    data_updated.SibSp=ss.fit_transform(data_updated.SibSp.values.reshape(-1,1))
    data_updated.Fare=ss.fit_transform(data_updated.Fare.fillna(data_updated.Fare.mean()).values.reshape(-1,1))
    data_updated.Age=ss.fit_transform(data_updated.Age.values.reshape(-1,1))
    return data_updated

In [5]:
processed_train=process_data(train)
processed_test=process_data(test)
processed_train_y=processed_train.Survived
processed_train_x=processed_train[processed_train.columns.drop('Survived')]
processed_train.head()

,Survived,Pclass,Name,SibSp,Fare,Embarked,Age,Sex,Cabin
0,0,3,-0.391348,0.059160,-0.502445,-0.581114,-0.102313,1,0.522067
1,1,1,1.635926,0.059160,0.786845,1.938460,0.807492,0,-1.917594
2,1,3,0.622289,-0.560975,-0.488854,-0.581114,0.125138,0,0.522067
3,1,1,1.635926,0.059160,0.420730,-0.581114,0.636903,0,-1.917594
4,0,3,-0.391348,-0.560975,-0.486337,-0.581114,0.636903,1,0.522067


In [6]:
#Plotting
#from sklearn.metrics import confusion_matrix
#confusion_matrix()
processed_train.corr()

,Survived,Pclass,Name,SibSp,Fare,Embarked,Age,Sex,Cabin
Survived,1.000000,-0.338481,0.402263,0.016639,0.257307,0.163517,0.010539,-0.543351,-0.301116
Pclass,-0.338481,1.000000,-0.071031,0.065997,-0.549500,-0.157112,-0.361353,0.131900,0.746616
Name,0.402263,-0.071031,1.000000,0.090879,0.129724,0.073551,0.010180,-0.778165,-0.092201
SibSp,0.016639,0.065997,0.090879,1.000000,0.217138,-0.064701,-0.150584,-0.200988,0.012131
Fare,0.257307,-0.549500,0.129724,0.217138,1.000000,0.221226,0.135516,-0.182333,-0.523013
Embarked,0.163517,-0.157112,0.073551,-0.064701,0.221226,1.000000,-0.087085,-0.104057,-0.187015
Age,0.010539,-0.361353,0.010180,-0.150584,0.135516,-0.087085,1.000000,0.024978,-0.274458
Sex,-0.543351,0.131900,-0.778165,-0.200988,-0.182333,-0.104057,0.024978,1.000000,0.123076
Cabin,-0.301116,0.746616,-0.092201,0.012131,-0.523013,-0.187015,-0.274458,0.123076,1.000000


In [7]:
# create model iterator
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.metrics import roc_auc_score
def model_check(est,train_x,train_y,test_x,test_y):
    model=rfc(n_estimators=est,random_state=rstate)
    model.fit(train_x,train_y)
    preds=model.predict(test_x)
    return roc_auc_score(test_y,preds)

In [8]:
#splitting data
def split(X,Y):
    from sklearn.model_selection import train_test_split
    return train_test_split(X,Y,test_size=t_size,random_state=rstate)

In [9]:
train_x,valid_x,train_y,valid_y=split(processed_train_x,processed_train_y)
mea_dict={}
for i in (100,200,300,400,500,550,600,650,750):
    mea_dict[i]=model_check(i,train_x,train_y,valid_x,valid_y)
print(mea_dict)

{100: 0.7832175925925925, 200: 0.7939814814814815, 300: 0.7908564814814815, 400: 0.7939814814814815, 500: 0.7939814814814815, 550: 0.7939814814814815, 600: 0.7954861111111111, 650: 0.7862268518518519, 750: 0.7893518518518519}


In [10]:
#Random Forest Model
model_rf=rfc(n_estimators=100,random_state=rstate)
model_rf.fit(processed_train_x,processed_train_y)

#preds=model_rf.predict(processed_test)
#out(preds)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=30, verbose=0,
                       warm_start=False)

In [11]:
#LightGBM Model
import lightgbm as lgb
train_x,valid_x,train_y,valid_y=split(processed_train_x,processed_train_y)
Dtrain=lgb.Dataset(train_x,train_y)
Dvalid=lgb.Dataset(valid_x,valid_y)
params={'num_leaves':'70',
        'max_depth':'7',
        'objective':'binary',
        'boosting':'rf',
        'bagging_freq':1,
        'bagging_fraction':'0.9',
        'metric': 'binary_logloss'}
gbm = lgb.train(params,
                Dtrain,
                num_boost_round=9,
                valid_sets=Dvalid  # eval training data
                )
preds_gbm=list(map(lambda x: 1 if x>0.5 else 0,gbm.predict(processed_test)))
#print(preds_gbm)
#out(preds_gbm)

[1]	valid_0's binary_logloss: 0.475961
[2]	valid_0's binary_logloss: 0.465167
[3]	valid_0's binary_logloss: 0.467138
[4]	valid_0's binary_logloss: 0.465885
[5]	valid_0's binary_logloss: 0.462099
[6]	valid_0's binary_logloss: 0.462972
[7]	valid_0's binary_logloss: 0.462057
[8]	valid_0's binary_logloss: 0.460788
[9]	valid_0's binary_logloss: 0.462444


In [12]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error as mea
model_nb=GaussianNB()
train_x,valid_x,train_y,valid_y=split(processed_train_x,processed_train_y)
model_nb.fit(train_x,train_y)
y_pred=model_nb.predict(valid_x)
print(roc_auc_score(y_pred,valid_y))
#model_nb.fit(processed_train_x,processed_train_y)
#preds_nb=model_nb.predict(processed_test)
#out(preds_nb)

0.7413369103509948


In [13]:
#XGB
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
model_xgb = XGBClassifier(n_estimators=180,random_state=rstate,learning_rate=0.01)
train_x,valid_x,train_y,valid_y=split(processed_train_x,processed_train_y)
valid_x,test_x,valid_y,test_y=split(valid_x,valid_y)
model_xgb.fit(train_x, train_y,early_stopping_rounds=15,eval_set=[(test_x,test_y)])
y_preds_x=model_xgb.predict(valid_x)
print(roc_auc_score(y_preds_x,valid_y))

#creating full model
train_x,valid_x,train_y,valid_y=split(processed_train_x,processed_train_y)
model_xgb.fit(train_x, train_y,early_stopping_rounds=15,eval_set=[(valid_x,valid_y)])
preds_xb=model_xgb.predict(processed_test)
out(preds_xb)

[0]	validation_0-error:0.197531
Will train until validation_0-error hasn't improved in 15 rounds.
[1]	validation_0-error:0.222222
[2]	validation_0-error:0.197531
[3]	validation_0-error:0.222222
[4]	validation_0-error:0.197531
[5]	validation_0-error:0.222222
[6]	validation_0-error:0.197531
[7]	validation_0-error:0.222222
[8]	validation_0-error:0.197531
[9]	validation_0-error:0.222222
[10]	validation_0-error:0.197531
[11]	validation_0-error:0.197531
[12]	validation_0-error:0.197531
[13]	validation_0-error:0.197531
[14]	validation_0-error:0.197531
[15]	validation_0-error:0.197531
Stopping. Best iteration:
[0]	validation_0-error:0.197531

0.7678703021370671
[0]	validation_0-error:0.216418
Will train until validation_0-error hasn't improved in 15 rounds.
[1]	validation_0-error:0.227612
[2]	validation_0-error:0.216418
[3]	validation_0-error:0.227612
[4]	validation_0-error:0.216418
[5]	validation_0-error:0.227612
[6]	validation_0-error:0.216418
[7]	validation_0-error:0.227612
[8]	validation_0

In [14]:
#MLP
from sklearn.neural_network import MLPClassifier
train_x,valid_x,train_y,valid_y=split(processed_train_x,processed_train_y)
model_MLP=MLPClassifier()
model_MLP.fit(train_x,train_y)
preds_mlp_y=model_MLP.predict(valid_x)
print(roc_auc_score(preds_mlp_y,valid_y))
print(model_MLP.score(valid_x,valid_y))

#predictions
preds_mlp=model_MLP.predict(processed_test)
out(preds_mlp)

0.7988764044943819
0.7985074626865671


/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [15]:
#KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
train_x,valid_x,train_y,valid_y=split(processed_train_x,processed_train_y)
model_KN=KNeighborsClassifier()
model_KN.fit(train_x,train_y)
preds_kn_y=model_KN.predict(valid_x)
print(roc_auc_score(preds_kn_y,valid_y))


0.7924731182795699
